# Leaderboards Analysis

Based on our current scouting, as well as The Blue Alliance's OPR score, and any other criteria we have, create a ranked list of teams within the competition.

*run all Data notebooks before any of the analysis notebooks.*

In [2]:
import bokeh
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn, HTMLTemplateFormatter
from bokeh.io import output_notebook, show

import glob
import logging

import numpy as np
import pandas as pd

import sys

output_notebook()
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logger = logging.getLogger('Leaderboard_Analysis')

from common import canalytics
from common import CLIMB_POINTS,CLIMB_NAMES,FIRST_AVG,LAST_AVG,TOTAL_AVG


Loading BokehJS ...

In [3]:
team_summary = pd.read_csv('./data/team_summary.csv', index_col=0)


# Let's look at the number of matches we have scouting data for.
# If there are more than FIRST_AVG + LAST_AVG, let's calculate a scouting score for the first 3 matches, and the most recent 3 matches.
# This will allow us to see if they had early issues that they resolved that might mean their OPR/Total event scouting score is not representative.
# Likewise if the most recent score was lower, then they may have some reliability issues with the robot.
# Also, determine the highest climb position.
team_summary.head()

,team,matches,scouted_matches,scouting_points_avg,climb_success_pct,climb_failure_pct,total_cargo_avg,auto_cargo_avg,teleop_cargo_avg,climb_points_avg,...,oprs,pre_oprs,pre_climb_points,pre_district_rank,pre_district_points,rank,wins,losses,ties,dq
0,418,12,7,13.000000,83.333333,0.000000,1.571429,0.714286,0.857143,8.666667,...,14.066553,12.40,9.00,103.0,19.0,6,8,4,0,0
1,457,12,8,5.875000,66.666667,25.000000,0.375000,0.375000,0.000000,4.750000,...,11.416101,10.12,10.83,84.0,24.0,10,6,5,1,0
2,1164,12,7,2.000000,33.333333,14.285714,0.142857,0.000000,0.142857,2.750000,...,5.891693,NaN,NaN,150.0,0.0,23,4,8,0,0
3,2158,12,6,26.000000,58.333333,33.333333,8.166667,0.833333,7.333333,6.500000,...,28.420253,27.19,8.25,55.0,33.0,9,6,5,1,0
4,2613,12,9,5.555556,0.000000,0.000000,4.777778,0.555556,4.222222,0.000000,...,7.869084,NaN,NaN,152.0,0.0,11,5,7,0,0


In [8]:
teams=team_summary[['team','rank','oprs','scouting_points_avg',"matches","highest_endgame_position","common_endgame_position","climb_success_pct"]]
teams=teams.round()

In [9]:
teams.to_csv("./data/leaderboard.csv")

In [16]:
source = ColumnDataSource(teams)

gocans_template="""                
            <p style="color:<%= 
                  (function colorfromteam(){
                    if (team == 2158 ){
                      return('#FFCF46')}
                    }()) %>;
                background-color:<%=
                  (function backgroundfromteam(){
                    if (team == 2158 ){
                      return('black')}
                    }()) %>;
                    "> 
                <%= value %>
            </p>
            """
team_formatter =  HTMLTemplateFormatter(template=gocans_template)

columns = [
        TableColumn(field="team", title="Team",width=50, formatter=team_formatter),
        TableColumn(field="rank", title="Rank",width=50 ),
        TableColumn(field="matches", title="Matches", width=75),
        TableColumn(field="highest_endgame_position", title="Max Climb",width=155),
        TableColumn(field="common_endgame_position", title="Common Climb",width=155),
        TableColumn(field="climb_success_pct", title="Climb Success Pct", width=175),
        TableColumn(field="oprs", title="OPR from TBA", width=175),
        TableColumn(field="scouting_points_avg", title="Average Scouting Points", width=200),
        #TableColumn(field="first_avg", title="Avg Scouting from first", width=200),
        #TableColumn(field="last_avg", title="Avg Scouting from last", width=200),
    ]

data_table = DataTable(source=source, columns=columns, width=1000,height=1200,
        sortable=True,
        editable=False,
        reorderable=True)

# Leaderboard

The table below shows their Offensive Power Rating Score from TBA, as well as the Average Scouting Score of the matches so far.
If our scouting is accurate, and the OPR is higher than our Average Scouting Score, then OPR might be higher than their overall value as an alliance partner.
If the OPR is lower than our average scouting score, then they may be a good alliance pick that may not be noticed by other teams.



In [17]:
show(data_table)